In [1]:
from typing import Tuple, List

import os
import dask.dataframe as dd
import numpy as np
import pandas as pd
import scipy.stats

In [ ]:
時間月份,日期,小時數,分鐘數,
週數,假日/非假日

In [2]:
log_root = r'D:\issp_data\pre_log'
out_root = r'D:\issp_data\nn'

In [3]:
func_table = {
    'max': np.max,
    'mean': np.mean,
    'median': np.median,
    'min': np.min,
    'std': np.std,
    'var': np.var,
    'kurtosis': scipy.stats.kurtosis,
    'skew': scipy.stats.skew,
    'len': len
}

In [4]:
def fe_statistics(func_table: dict,
                  prefix: str, ds: 'pd.Series') -> 'pd.DataFrame':
    ds = ds.map(lambda ts: eval(ts))
    ret = pd.DataFrame()
    
    for k, func in func_table.items():
        print(prefix + k)
        ret[prefix + k] = ds.map(lambda ts: func_table[k](ts))
        
    return ret


def feature_extraction_login(in_path: str, out_root: str) -> None:
    fl_nm = in_path.split('\\')[-1][:-4] + 'logfe.csv'
    print(fl_nm)
    
    df = dd.read_csv(in_path, compression='zip')\
            .compute()
    df_ts = fe_statistics(func_table, r'ts_', df['Timestamp'])
    df_dr = fe_statistics(func_table, r'dr_', df['Duration'])
    df.drop(columns=['Timestamp', 'Duration'], inplace=True)
    
    pd.concat([df, df_ts, df_dr], axis=1)\
        .to_csv(os.path.join(out_root, fl_nm), 
                header=True, index=None, compression='zip')

In [5]:
file_li = [fl for fl in os.listdir(log_root) if 'os' not in fl]
file_li = file_li[15:]

for fl in file_li:
    path = os.path.join(log_root, fl)
    feature_extraction_login(path, out_root)

s35809_MobileApp_201706_104649_0logfe.csv


C:\Anaconda3\lib\site-packages\dask\dataframe\io\csv.py:385: UserWarning: Warning zip compression does not support breaking apart files
Please ensure that each individual file can fit in memory and
use the keyword ``blocksize=None to remove this message``
Setting ``blocksize=None``
  "Setting ``blocksize=None``" % compression


KeyboardInterrupt: 

In [ ]:
path = r'D:\issp_data\fe_log\s35809_MobileApp_201701_104628_0logfe.csv'
df = dd.read_csv(path, compression='zip')\
        .compute()
df